In [4]:
%%writefile myThread.py
#!/usr/bin/env python

import threading
from time import ctime

class MyThread(threading.Thread):
    
    def __init__(self, func, args, name=''):
        threading.Thread.__init__(self)
        self.name = name
        self.func = func
        self.args = args
        
    def getResult(self):
        return self.res
        
    def run(self):
        print('staring {} at: {}'.format(self.name, ctime()))
        self.res = self.func(*self.args)
        print('{} finished at: {}'.format(self.name, ctime()))

Overwriting myThread.py


## Comparing Single vs. Multithreaded Execution

In [7]:
#!/usr/bin/env python 
from myThread import MyThread
from time import ctime, sleep

def fib(x):
    sleep(0.005)
    if x < 2: return 1
    return (fib(x-2) + fib(x-1))

def fac(x):
    sleep(0.1)
    if x < 2: return 1 
    return (x * fac(x-1))
The 
def sum(x):
    sleep(0.1)
    if x < 2: return 1 
    return (x + sum(x-1))

funcs = [fib, fac, sum]
n = 12

def main():
    nfuncs = list(range(len(funcs)))
    print('*** SINGLE THREAD') 
    for i in nfuncs:
        print('starting', funcs[i].__name__, 'at:', ctime())
        print(funcs[i](n))
        print(funcs[i].__name__, 'finished at:', ctime())
    print('\n*** MULTIPLE THREADS')
    threads = []
    for i in nfuncs:
        t = MyThread(funcs[i], (n,), funcs[i].__name__)
        threads.append(t)
    
    for i in nfuncs:
        threads[i].start()
    
    for i in nfuncs:
        threads[i].join()
        print(threads[i].getResult())
    
    print('all DONE')

if __name__ == '__main__':
    main()

*** SINGLE THREAD
starting fib at: Sat May 25 12:08:06 2019
233
fib finished at: Sat May 25 12:08:08 2019
starting fac at: Sat May 25 12:08:08 2019
479001600
fac finished at: Sat May 25 12:08:09 2019
starting sum at: Sat May 25 12:08:09 2019
78
sum finished at: Sat May 25 12:08:11 2019

*** MULTIPLE THREADS
staring fib at: Sat May 25 12:08:11 2019staring fac at: Sat May 25 12:08:11 2019

staring sum at: Sat May 25 12:08:11 2019
sum finished at: Sat May 25 12:08:12 2019
fac finished at: Sat May 25 12:08:12 2019
fib finished at: Sat May 25 12:08:13 2019
233
479001600
78
all DONE


## Example: Book Rankings "Screenscraper"(bookrank.py)

In [22]:
#!/usr/bin/env python
# coding:utf-8
from atexit import register
from re import compile
from threading import Thread
from time import ctime

import requests

REGEX = compile('#([\d,]+) in Books')
AMZN = 'https://amazon.com/dp/'
ISBNs = {
    '0132269937': 'Core Python Programming',
    '0132356139': 'Python Web Development with Django',
    '0137143419': 'Python Fundamentals',
}

def getRanking(isbn):
    _url = '%s%s' % (AMZN, isbn)
    user_agent = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
    headers = {'User-Agent': user_agent}
    page = requests.get(_url, headers=headers)
    data = page.text
    if page.status_code == 200:
        return REGEX.findall(data)[0]
    else:
        return "unknown"

def _showRanking(isbn):
    print('- %r randked %s' %(ISBNs[isbn], getRanking(isbn)))

def _main():
    print('At', ctime(), 'on Amazon...')
    for isbn in ISBNs:
       # _showRanking(isbn)
       Thread(target=_showRanking, args=(isbn,)).start()

@register # atexit模块主要的作用就是在程序即将结束之前执行的代码, atexit.register 注册函数
def _atexit():
    print('all DONE at:', ctime())


if __name__ == '__main__':
    _main()

At Sun May 26 10:37:40 2019 on Amazon...
- 'Python Fundamentals' randked 5,299,974
- 'Core Python Programming' randked 803,367
- 'Python Web Development with Django' randked 372,528


## Locks and More Randomness (mtsleepF.py)

In [23]:
#!/usr/bin/env python
from atexit import register
from random import randrange
from threading import Thread, currentThread, Lock
from time import sleep, ctime


class CleanOutputSet(set):
    def __str__(self):
        return ', '.join(x for x in self)


loops = (randrange(2, 5) for x in range(randrange(3, 7)))
remaining = CleanOutputSet()
lock = Lock()

def loop_without_lock(nsec):
    myname = currentThread().name
    remaining.add(myname)
    print('[%s] Started %s' % (ctime(), myname))
    sleep(nsec)
    remaining.remove(myname)
    print('[%s] Completed %s (%d secs)' % (ctime(), myname, nsec))
    print(' (remaining: %s)' % (remaining or 'NONE'))

def loop(nsec):
    myname = currentThread().name
    with lock:
        remaining.add(myname)
        print('[%s] Started %s' % (ctime(), myname))
    sleep(nsec)
    with lock:    # 等价于第一句添加lock.acquire(),最后一句添加lock.release()
        remaining.remove(myname)
        print('[%s] Completed %s (%d secs)' % (ctime(), myname, nsec))
        print(' (remaining: %s)' % (remaining or 'NONE'))


def _main():
    for pause in loops:
        Thread(target=loop, args=(pause,)).start()

    # for pause in loops:
    #     Thread(target=loop_without_lock, args=(pause,)).start()


@register
def _atexit():
    print('all DONE at:', ctime())

if __name__ == '__main__':
    _main()


[Sun May 26 11:38:05 2019] Started Thread-34
[Sun May 26 11:38:05 2019] Started Thread-35
[Sun May 26 11:38:05 2019] Started Thread-36
[Sun May 26 11:38:05 2019] Started Thread-37
[Sun May 26 11:38:05 2019] Started Thread-38
[Sun May 26 11:38:07 2019] Completed Thread-37 (2 secs)
 (remaining: Thread-35, Thread-36, Thread-38, Thread-34)
[Sun May 26 11:38:08 2019] Completed Thread-34 (3 secs)
 (remaining: Thread-35, Thread-36, Thread-38)
[Sun May 26 11:38:08 2019] Completed Thread-38 (3 secs)
 (remaining: Thread-35, Thread-36)
[Sun May 26 11:38:09 2019] Completed Thread-35 (4 secs)
 (remaining: Thread-36)
[Sun May 26 11:38:09 2019] Completed Thread-36 (4 secs)
 (remaining: NONE)
